Here we'll first be loading the FC matrices and explore their structure

In [1]:
#using read_dataset from Datasets/FC/create_dataset.py to read the dataset
from Datasets.FC.create_dataset import read_dataset_MDD
import networkx as nx

In [2]:
dataset = read_dataset_MDD()

In [3]:
#ok, let's explore the data a bit more
#dataset is a list object of torch_geometric.data objects

#let's see the first element
print(dataset[0])

#it's a dictionary object, let's see the keys
print(dataset[0].keys())

#ok, let's see the values
print(dataset[0].values())

#it has 4 keys, 'x', 'edge_index', 'edge_attr' and 'y' where y=0 menas the patient is healthy and y=1 means the patient has Autism Spectrum Disorder (ASD)
"""graph = Data(x=ROI.reshape(-1,116).float(),
                     edge_index=G.indices().reshape(2,-1).long(),
                     edge_attr=G.values().reshape(-1,1).float(),
                     y=y.long())"""

#how much data do we have?
len(dataset)


Data(x=[116, 116], edge_index=[2, 1294], edge_attr=[1294, 1], y=[1])
['edge_index', 'edge_attr', 'y', 'x']
ValuesView({'x': tensor([[ 0.0000,  0.2857,  0.0804,  ...,  0.2032,  0.1674,  0.0906],
        [ 0.2857,  0.0000, -0.3860,  ...,  0.1637, -0.0359,  0.1674],
        [ 0.0804, -0.3860,  0.0000,  ..., -0.0175, -0.0309,  0.0296],
        ...,
        [ 0.2032,  0.1637, -0.0175,  ...,  0.0000,  0.2329, -0.1452],
        [ 0.1674, -0.0359, -0.0309,  ...,  0.2329,  0.0000,  0.0183],
        [ 0.0906,  0.1674,  0.0296,  ..., -0.1452,  0.0183,  0.0000]]), 'edge_index': tensor([[  0,   0,   0,  ..., 113, 113, 114],
        [ 10,  12,  14,  ..., 111, 112, 108]]), 'edge_attr': tensor([[0.7785],
        [0.6966],
        [0.5463],
        ...,
        [0.6415],
        [0.4745],
        [0.5401]]), 'y': tensor([0])})


1604

In [4]:
#set the seed
import torch
torch.manual_seed(0)

MODEL = "GAT"
DATASET = "FC_MDD"

from models.models_FC import GAT_framework as framework # import the model

gnn = framework(dataset)

print(gnn.model)
print(gnn.train_idx)

Net(
  (conv_layers): ModuleList(
    (0): GATConv(116, 128, heads=4)
    (1): GATConv(512, 128, heads=4)
    (2): GATConv(512, 128, heads=4)
    (3): GATConv(512, 128, heads=4)
    (4): GATConv(512, 128, heads=4)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm(512)
    (1): BatchNorm(512)
    (2): BatchNorm(512)
    (3): BatchNorm(512)
    (4): BatchNorm(512)
  )
  (lin1): Linear(512, 128, bias=True)
  (lin2): Linear(128, 2, bias=True)
  (bn1): BatchNorm(128)
  (bn2): BatchNorm(2)
)
tensor([1365,  984, 1017,  ...,  260,  678,   83])


/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
gnn.iterate()

/home/tpelletreaudur/.local/lib/python3.10/site-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 005, Loss: 0.158, Test Loss: 1.039, Train Acc: 0.485 Test Acc: 0.481
Epoch: 010, Loss: 0.123, Test Loss: 0.646, Train Acc: 1.000 Test Acc: 0.642
Epoch: 015, Loss: 0.115, Test Loss: 0.664, Train Acc: 1.000 Test Acc: 0.642
Epoch: 020, Loss: 0.107, Test Loss: 0.660, Train Acc: 1.000 Test Acc: 0.630
Epoch: 025, Loss: 0.101, Test Loss: 0.664, Train Acc: 1.000 Test Acc: 0.605
Epoch: 030, Loss: 0.097, Test Loss: 0.650, Train Acc: 1.000 Test Acc: 0.630
Epoch: 035, Loss: 0.093, Test Loss: 0.664, Train Acc: 1.000 Test Acc: 0.580
Epoch: 040, Loss: 0.088, Test Loss: 0.668, Train Acc: 1.000 Test Acc: 0.605
Epoch: 045, Loss: 0.084, Test Loss: 0.670, Train Acc: 1.000 Test Acc: 0.605
Epoch: 050, Loss: 0.080, Test Loss: 0.666, Train Acc: 1.000 Test Acc: 0.580
Epoch: 055, Loss: 0.076, Test Loss: 0.658, Train Acc: 1.000 Test Acc: 0.617
Epoch: 060, Loss: 0.073, Test Loss: 0.661, Train Acc: 1.000 Test Acc: 0.593
Epoch: 065, Loss: 0.070, Test Loss: 0.647, Train Acc: 1.000 Test Acc: 0.654
Epoch: 070, 

In [6]:
#gnn.train()

In [7]:
#save the model 
gnn.save_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

Model saved in: models/FC_MDD_GATserver.pt


In [6]:
#load the model
gnn.load_model(path="models/"+DATASET+"_"+MODEL+"server.pt")

In [8]:
gnn.evaluate()

Test Loss: 0.635, Train Acc: 1.000 Test Acc: 0.667


In [9]:
train_features, test_features = gnn.evaluate_with_features2()

In [10]:
print(len(train_features[0]))
len(train_features), len(test_features)

8


(1523, 81)

In [11]:
import networkx as nx
def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Alternative metrics for disconnected graphs
        # Option 1: Use the average path length of the largest connected component
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        density = nx.density(G)
        avg_path_len = calculate_avg_path_length(G)
        num_cliques = len(list(nx.find_cliques(G)))
        num_triangles = sum(nx.triangles(G).values()) / 3
        num_squares = sum(nx.square_clustering(G).values()) / 4
        number_of_node_in_the_largest_fully_connected_component = len(max(nx.connected_components(G), key=len))
        #small_world = nx.algorithms.smallworld.sigma(G)

        properties.append((num_nodes, num_edges, density, avg_path_len, num_cliques, num_triangles, num_squares, number_of_node_in_the_largest_fully_connected_component)) #, small_world))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties = compute_graph_properties(selected_dataset)

In [12]:
print(len(train_properties))
train_properties[0:5]



1523


[(116,
  488,
  0.07316341829085457,
  3.314842578710645,
  148,
  938.0,
  6.439699892510922,
  116),
 (114,
  598,
  0.0928427262847384,
  3.166744294364229,
  164,
  1378.0,
  7.519630514954544,
  114),
 (114,
  408,
  0.06334420121099209,
  3.391864617295451,
  135,
  511.0,
  5.279502013122494,
  114),
 (115,
  744,
  0.11350114416475973,
  2.796796338672769,
  198,
  1870.0,
  7.980704583203741,
  115),
 (112,
  706,
  0.11357786357786358,
  3.3745173745173744,
  181,
  2528.0,
  9.050100863174576,
  112)]

In [13]:
import pickle as pkl
#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "wb") as f:
    pkl.dump(train_properties, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "wb") as f:
    pkl.dump(test_properties, f)

#load the properties
with open("results/"+DATASET+"_"+MODEL+"_train_properties.pkl", "rb") as f:
    train_properties = pkl.load(f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties.pkl", "rb") as f:
    test_properties = pkl.load(f)

In [14]:
"""
The embeddings of GCN are like this:
return F.log_softmax(x7, dim=-1), (x1, x2, x3, x4, x5, x_global, x6, x7)
"""


import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
output_size = 1  # Predicting one property at a time
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Assume we have already evaluated to get features
# train_features, test_features = gnn.evaluate_with_features2()

# Extract x embeddings
train_x = np.array([feat[0] for feat in train_features])
test_x = np.array([feat[0] for feat in test_features])

# Extract 2, 3, 4, global, 5, 6, 7 embeddings
train_x2 = np.array([feat[1] for feat in train_features])
test_x2 = np.array([feat[1] for feat in test_features])

train_x3 = np.array([feat[2] for feat in train_features])
test_x3 = np.array([feat[2] for feat in test_features])

train_x4 = np.array([feat[3] for feat in train_features])
test_x4 = np.array([feat[3] for feat in test_features])

train_x5 = np.array([feat[4] for feat in train_features])
test_x5 = np.array([feat[4] for feat in test_features])

train_x_global = np.array([feat[5] for feat in train_features])
test_x_global = np.array([feat[5] for feat in test_features])

train_x6 = np.array([feat[6] for feat in train_features])
test_x6 = np.array([feat[6] for feat in test_features])

train_x7 = np.array([feat[7] for feat in train_features])
test_x7 = np.array([feat[7] for feat in test_features])

# Compute graph properties
# train_properties = compute_graph_properties(gnn.dataset[gnn.train_idx])
# test_properties = compute_graph_properties(gnn.dataset[gnn.test_idx])

# Convert to PyTorch tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_x2 = torch.tensor(train_x2, dtype=torch.float32)
train_x3 = torch.tensor(train_x3, dtype=torch.float32)
train_x4 = torch.tensor(train_x4, dtype=torch.float32)
train_x5 = torch.tensor(train_x5, dtype=torch.float32)
train_x_global = torch.tensor(train_x_global, dtype=torch.float32)
train_x6 = torch.tensor(train_x6, dtype=torch.float32)
train_x7 = torch.tensor(train_x7, dtype=torch.float32)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_x2 = torch.tensor(test_x2, dtype=torch.float32)
test_x3 = torch.tensor(test_x3, dtype=torch.float32)
test_x4 = torch.tensor(test_x4, dtype=torch.float32)
test_x5 = torch.tensor(test_x5, dtype=torch.float32)
test_x_global = torch.tensor(test_x_global, dtype=torch.float32)
test_x6 = torch.tensor(test_x6, dtype=torch.float32)
test_x7 = torch.tensor(test_x7, dtype=torch.float32)

train_y = torch.tensor(train_properties, dtype=torch.float32)
test_y = torch.tensor(test_properties, dtype=torch.float32)

# Train and evaluate a model for each graph property and each embedding
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings = [(train_x, test_x), (train_x2, test_x2), (train_x3, test_x3), (train_x4, test_x4), (train_x5, test_x5), (train_x_global, test_x_global), (train_x6, test_x6), (train_x7, test_x7)]
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y[:, i].cpu().numpy()
            test_target = test_y[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "wb") as f:
    pkl.dump(results, f)

Epoch [1000/800000], Property: num_nodes, Loss: 8624.2012
Epoch [2000/800000], Property: num_nodes, Loss: 5942.1797
Epoch [3000/800000], Property: num_nodes, Loss: 4214.9302
Epoch [4000/800000], Property: num_nodes, Loss: 3075.3374
Epoch [5000/800000], Property: num_nodes, Loss: 2299.4534
Epoch [6000/800000], Property: num_nodes, Loss: 1759.1758
Epoch [7000/800000], Property: num_nodes, Loss: 1380.2865
Epoch [8000/800000], Property: num_nodes, Loss: 1111.2935
Epoch [9000/800000], Property: num_nodes, Loss: 915.1191
Epoch [10000/800000], Property: num_nodes, Loss: 767.9552
Epoch [11000/800000], Property: num_nodes, Loss: 655.2929
Epoch [12000/800000], Property: num_nodes, Loss: 567.0733
Epoch [13000/800000], Property: num_nodes, Loss: 496.3718
Epoch [14000/800000], Property: num_nodes, Loss: 439.2293
Epoch [15000/800000], Property: num_nodes, Loss: 393.2050
Epoch [16000/800000], Property: num_nodes, Loss: 356.0715
Epoch [17000/800000], Property: num_nodes, Loss: 325.8481
Epoch [18000/80

Epoch [11000/800000], Property: num_edges, Loss: 111468.9219
Epoch [12000/800000], Property: num_edges, Loss: 103131.8359
Epoch [13000/800000], Property: num_edges, Loss: 95575.0000
Epoch [14000/800000], Property: num_edges, Loss: 88720.9766
Epoch [15000/800000], Property: num_edges, Loss: 82493.2656
Epoch [16000/800000], Property: num_edges, Loss: 76831.8672
Epoch [17000/800000], Property: num_edges, Loss: 71687.8047
Epoch [18000/800000], Property: num_edges, Loss: 67023.4609
Epoch [19000/800000], Property: num_edges, Loss: 62805.1055
Epoch [20000/800000], Property: num_edges, Loss: 58999.4219
Epoch [21000/800000], Property: num_edges, Loss: 55567.4531
Epoch [22000/800000], Property: num_edges, Loss: 52464.3086
Epoch [23000/800000], Property: num_edges, Loss: 49645.8047
Epoch [24000/800000], Property: num_edges, Loss: 47074.6094
Epoch [25000/800000], Property: num_edges, Loss: 44724.6836
Epoch [26000/800000], Property: num_edges, Loss: 42581.1719
Epoch [27000/800000], Property: num_ed

Epoch [147000/800000], Property: num_edges, Loss: 12501.8018
Epoch [148000/800000], Property: num_edges, Loss: 12482.1514
Epoch [149000/800000], Property: num_edges, Loss: 12462.6240
Epoch [150000/800000], Property: num_edges, Loss: 12443.2178
Epoch [151000/800000], Property: num_edges, Loss: 12423.9277
Epoch [152000/800000], Property: num_edges, Loss: 12404.7607
Epoch [153000/800000], Property: num_edges, Loss: 12385.7129
Epoch [154000/800000], Property: num_edges, Loss: 12366.7803
Epoch [155000/800000], Property: num_edges, Loss: 12347.9648
Epoch [156000/800000], Property: num_edges, Loss: 12329.2637
Epoch [157000/800000], Property: num_edges, Loss: 12310.6777
Epoch [158000/800000], Property: num_edges, Loss: 12292.2109
Epoch [159000/800000], Property: num_edges, Loss: 12273.8584
Epoch [160000/800000], Property: num_edges, Loss: 12255.6191
Epoch [161000/800000], Property: num_edges, Loss: 12237.4980
Epoch [162000/800000], Property: num_edges, Loss: 12219.4863
Epoch [163000/800000], P

Epoch [282000/800000], Property: num_edges, Loss: 10455.6133
Epoch [283000/800000], Property: num_edges, Loss: 10443.5840
Epoch [284000/800000], Property: num_edges, Loss: 10431.6045
Epoch [285000/800000], Property: num_edges, Loss: 10419.6699
Epoch [286000/800000], Property: num_edges, Loss: 10407.7842
Epoch [287000/800000], Property: num_edges, Loss: 10395.9434
Epoch [288000/800000], Property: num_edges, Loss: 10384.1514
Epoch [289000/800000], Property: num_edges, Loss: 10372.4072
Epoch [290000/800000], Property: num_edges, Loss: 10360.7090
Epoch [291000/800000], Property: num_edges, Loss: 10349.0605
Epoch [292000/800000], Property: num_edges, Loss: 10337.4580
Epoch [293000/800000], Property: num_edges, Loss: 10325.9043
Epoch [294000/800000], Property: num_edges, Loss: 10314.3965
Epoch [295000/800000], Property: num_edges, Loss: 10302.9355
Epoch [296000/800000], Property: num_edges, Loss: 10291.5244
Epoch [297000/800000], Property: num_edges, Loss: 10280.1592
Epoch [298000/800000], P

Epoch [418000/800000], Property: num_edges, Loss: 9255.2520
Epoch [419000/800000], Property: num_edges, Loss: 9249.8291
Epoch [420000/800000], Property: num_edges, Loss: 9244.4619
Epoch [421000/800000], Property: num_edges, Loss: 9239.1387
Epoch [422000/800000], Property: num_edges, Loss: 9233.8574
Epoch [423000/800000], Property: num_edges, Loss: 9228.6230
Epoch [424000/800000], Property: num_edges, Loss: 9223.4316
Epoch [425000/800000], Property: num_edges, Loss: 9218.2871
Epoch [426000/800000], Property: num_edges, Loss: 9213.1865
Epoch [427000/800000], Property: num_edges, Loss: 9208.1279
Epoch [428000/800000], Property: num_edges, Loss: 9203.1172
Epoch [429000/800000], Property: num_edges, Loss: 9198.1494
Epoch [430000/800000], Property: num_edges, Loss: 9193.2266
Epoch [431000/800000], Property: num_edges, Loss: 9188.3477
Epoch [432000/800000], Property: num_edges, Loss: 9183.5137
Epoch [433000/800000], Property: num_edges, Loss: 9178.7246
Epoch [434000/800000], Property: num_edg

...

Epoch [545000/800000], Property: num_triangles, Loss: 568248.1250
Epoch [546000/800000], Property: num_triangles, Loss: 567342.5000
Epoch [547000/800000], Property: num_triangles, Loss: 566438.7500
Epoch [548000/800000], Property: num_triangles, Loss: 565537.0000
Epoch [549000/800000], Property: num_triangles, Loss: 564637.2500
Epoch [550000/800000], Property: num_triangles, Loss: 563739.4375
Epoch [551000/800000], Property: num_triangles, Loss: 562843.6250
Epoch [552000/800000], Property: num_triangles, Loss: 561949.7500
Epoch [553000/800000], Property: num_triangles, Loss: 561057.8750
Epoch [554000/800000], Property: num_triangles, Loss: 560167.9375
Epoch [555000/800000], Property: num_triangles, Loss: 559279.8750
Epoch [556000/800000], Property: num_triangles, Loss: 558393.9375
Epoch [557000/800000], Property: num_triangles, Loss: 557509.9375
Epoch [558000/800000], Property: num_triangles, Loss: 556627.8125
Epoch [559000/800000], Property: num_triangles, Loss: 555747.6875
Epoch [560

Epoch [670000/800000], Property: num_triangles, Loss: 470257.9688
Epoch [671000/800000], Property: num_triangles, Loss: 469596.7500
Epoch [672000/800000], Property: num_triangles, Loss: 468937.4688
Epoch [673000/800000], Property: num_triangles, Loss: 468280.0625
Epoch [674000/800000], Property: num_triangles, Loss: 467624.5000
Epoch [675000/800000], Property: num_triangles, Loss: 466970.8125
Epoch [676000/800000], Property: num_triangles, Loss: 466319.0000
Epoch [677000/800000], Property: num_triangles, Loss: 465668.9688
Epoch [678000/800000], Property: num_triangles, Loss: 465020.8438
Epoch [679000/800000], Property: num_triangles, Loss: 464374.6250
Epoch [680000/800000], Property: num_triangles, Loss: 463730.2500
Epoch [681000/800000], Property: num_triangles, Loss: 463087.7500
Epoch [682000/800000], Property: num_triangles, Loss: 462447.0625
Epoch [683000/800000], Property: num_triangles, Loss: 461808.2812
Epoch [684000/800000], Property: num_triangles, Loss: 461171.3438
Epoch [685

Epoch [795000/800000], Property: num_triangles, Loss: 402053.3750
Epoch [796000/800000], Property: num_triangles, Loss: 401625.1250
Epoch [797000/800000], Property: num_triangles, Loss: 401198.7188
Epoch [798000/800000], Property: num_triangles, Loss: 400774.2188
Epoch [799000/800000], Property: num_triangles, Loss: 400351.5938
Epoch [800000/800000], Property: num_triangles, Loss: 399930.7812
Embedding: torch.Size([1523, 2])
Property: num_triangles
  Train MSE: 399930.3750, Test MSE: 591220.4375
  Train R²: -0.1371, Test R²: -0.1953
Epoch [1000/800000], Property: num_squares, Loss: 22.6113
Epoch [2000/800000], Property: num_squares, Loss: 15.1854
Epoch [3000/800000], Property: num_squares, Loss: 9.7304
Epoch [4000/800000], Property: num_squares, Loss: 5.9264
Epoch [5000/800000], Property: num_squares, Loss: 3.5010
Epoch [6000/800000], Property: num_squares, Loss: 2.1672
Epoch [7000/800000], Property: num_squares, Loss: 1.5937
Epoch [8000/800000], Property: num_squares, Loss: 1.4329
Epo

Epoch [65000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 2368.4155
Epoch [66000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 2274.3735
Epoch [67000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 2182.4226
Epoch [68000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 2092.4351
Epoch [69000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 2004.4106
Epoch [70000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1918.3501
Epoch [71000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1834.2524
Epoch [72000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1752.1172
Epoch [73000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 1671.9443
Epoch [74000/800000], Property: number_of_node

Epoch [143000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.7519
Epoch [144000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.6344
Epoch [145000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.5272
Epoch [146000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.4300
Epoch [147000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.3428
Epoch [148000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.2655
Epoch [149000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.1979
Epoch [150000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.1399
Epoch [151000/800000], Property: number_of_nodes_in_the_largest_fully_connected_component, Loss: 12.0911
Epoch [152000/800000], Property: number_of_nodes_in_the

In [ ]:
#load results
with open("results/"+DATASET+"_"+MODEL+"_results.pkl", "rb") as f:
    results = pkl.load(f)

In [16]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_name)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save
plt.savefig("results/"+DATASET+"_"+MODEL+"test_R2.png")


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import matplotlib.pyplot as plt

# Assuming results, embeddings, and other necessary variables are defined as in your context
property_names = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'num_cliques', 'num_triangles', 'num_squares', 'number_of_nodes_in_the_largest_fully_connected_component']
embeddings_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_global', 'x6', 'x7']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

plt.figure(figsize=(12, 8))

for i, property_name in enumerate(property_names):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_name)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_name, color=colors[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/'+DATASET+'_'+MODEL+'train_R2.png')


#### Test with more properties

In [ ]:
import networkx as nx
import numpy as np

def calculate_avg_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # Use the average path length of the largest connected component for disconnected graphs
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
        largest_component = max(components, key=len)
        return nx.average_shortest_path_length(largest_component)
    
def betweenness_centralization(G):
    n = len(G)
    betweenness = nx.betweenness_centrality(G)
    max_betweenness = max(betweenness.values())
    centralization = sum(max_betweenness - bet for bet in betweenness.values())
    if n > 2:
        centralization /= (n - 1) * (n - 2) / 2
    return centralization

def pagerank_centralization(G, alpha=0.85):
    n = len(G)
    pagerank = nx.pagerank(G, alpha=alpha)
    max_pagerank = max(pagerank.values())
    centralization = sum(max_pagerank - pr for pr in pagerank.values())
    if n > 1:
        centralization /= (n - 1)
    return centralization

def clustering_properties(G):
    average_clustering = nx.average_clustering(G)
    transitivity = nx.transitivity(G)
    return average_clustering, transitivity

def compute_graph_properties(data):
    properties = []
    for graph_data in data:
        G = nx.from_edgelist(graph_data.edge_index.t().tolist())
        
        # Number of nodes
        num_nodes = G.number_of_nodes()
        
        # Number of edges
        num_edges = G.number_of_edges()
        
        # Density
        density = nx.density(G)
        
        # Average Path Length
        avg_path_len = calculate_avg_path_length(G)
        
        # Diameter
        if nx.is_connected(G):
            diameter = nx.diameter(G)
        else:
            # Use the diameter of the largest connected component for disconnected graphs
            components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
            largest_component = max(components, key=len)
            diameter = nx.diameter(largest_component)
        
        # Radius
        if nx.is_connected(G):
            radius = nx.radius(G)
        else:
            radius = nx.radius(largest_component)
        
        # Clustering Coefficient
        clustering_coeff = nx.average_clustering(G)
        
        # Transitivity
        transitivity = nx.transitivity(G)
        
        # Assortativity
        assortativity = nx.degree_assortativity_coefficient(G)
        
        # Number of Cliques
        num_cliques = len(list(nx.find_cliques(G)))
        
        # Number of Triangles
        num_triangles = sum(nx.triangles(G).values()) / 3
        
        # Number of Squares (4-cycles)
        num_squares = sum(nx.square_clustering(G).values()) / 4
        
        # Size of the Largest Connected Component
        largest_component_size = len(max(nx.connected_components(G), key=len))
        
        # Average Degree
        degrees = [d for n, d in G.degree()]
        avg_degree = np.mean(degrees)
        
        # Betweenness Centrality
        betweenness_centrality = nx.betweenness_centrality(G)
        avg_betweenness_centrality = np.mean(list(betweenness_centrality.values()))
        
        # Eigenvalues of the Adjacency Matrix (for spectral properties)
        eigenvalues = np.linalg.eigvals(nx.adjacency_matrix(G).todense())
        spectral_radius = max(eigenvalues)
        algebraic_connectivity = sorted(eigenvalues)[1]  # second smallest eigenvalue
        
        # Graph Laplacian Eigenvalues
        laplacian_eigenvalues = np.linalg.eigvals(nx.laplacian_matrix(G).todense())
        graph_energy = sum(abs(laplacian_eigenvalues))
        
        # Small-World-ness
        # Compare clustering coefficient and average path length with those of a random graph
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)
        random_clustering_coeff = nx.average_clustering(random_graph)
        random_avg_path_len = calculate_avg_path_length(random_graph)
        small_world_coefficient = (clustering_coeff / random_clustering_coeff) / (avg_path_len / random_avg_path_len)

        # Calculate Betweenness Centralization
        betweenness_cent = betweenness_centralization(G)
        print(f"Betweenness Centralization: {betweenness_cent}")

        # Calculate PageRank Centralization
        pagerank_cent = pagerank_centralization(G)
        print(f"PageRank Centralization: {pagerank_cent}")

        # Calculate Clustering properties
        avg_clustering, transitivity = clustering_properties(G)
        print(f"Average Clustering Coefficient: {avg_clustering}")
        print(f"Transitivity: {transitivity}")
        
        properties.append((
            num_nodes,
            num_edges,
            density,
            avg_path_len,
            diameter,
            radius,
            clustering_coeff,
            transitivity,
            assortativity,
            num_cliques,
            num_triangles,
            num_squares,
            largest_component_size,
            avg_degree,
            avg_betweenness_centrality,
            spectral_radius,
            algebraic_connectivity,
            graph_energy,
            small_world_coefficient
        ))
    return properties

train_idx_list = gnn.train_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in train_idx_list]
train_properties_long = compute_graph_properties(selected_dataset)
test_idx_list = gnn.test_idx.tolist()
selected_dataset = [gnn.dataset[i] for i in test_idx_list]
test_properties_long = compute_graph_properties(selected_dataset)

#save the properties in a file
with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "wb") as f:
    pkl.dump(train_properties_long, f)

with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "wb") as f:
    pkl.dump(test_properties_long, f)

#load the properties
# with open("results/"+DATASET+"_"+MODEL+"_train_properties_long.pkl", "rb") as f:
#     train_properties_long = pkl.load(f)

# with open("results/"+DATASET+"_"+MODEL+"_test_properties_long.pkl", "rb") as f:
#     test_properties_long = pkl.load(f)

In [ ]:
property_names_long = ['num_nodes', 'num_edges', 'density', 'avg_path_len', 'diameter', 'radius', 'clustering_coeff', 'transitivity', 'assortativity', 'num_cliques', 'num_triangles', 'num_squares', 'largest_component_size', 'avg_degree', 'avg_betweenness_centrality', 'spectral_radius', 'algebraic_connectivity', 'graph_energy', 'small_world_coefficient']
train_y_long = torch.tensor(train_properties_long, dtype=torch.float32)
test_y_long = torch.tensor(test_properties_long, dtype=torch.float32)
#create a dictionary where we will store the results for each embeddings, each property
results = {}

ii = 0

for train_embedding, test_embedding in embeddings:
    input_size = train_embedding.shape[1]

    for i, property_name in enumerate(property_names_long):
        model = LinearModel(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        num_epochs = 800000  # Maximum number of epochs
        min_epochs = 1000  # Minimum number of epochs
        patience = 3000  # Number of epochs to wait for improvement
        tolerance = 1e-6  # Tolerance for considering the loss as stable

        best_loss = float('inf')
        no_improve_count = 0
        
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()

            outputs = model(train_embedding).squeeze()
            target = train_y_long[:, i].squeeze()

            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:  # Print every 1000 epochs
                print(f'Epoch [{epoch+1}/{num_epochs}], Property: {property_name}, Loss: {loss.item():.4f}')

            # Check for early stopping, but only after minimum epochs
            if epoch >= min_epochs:
                if loss.item() < best_loss - tolerance:
                    best_loss = loss.item()
                    no_improve_count = 0
                else:
                    no_improve_count += 1

                if no_improve_count >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_pred = model(train_embedding).squeeze().cpu().numpy()
            test_pred = model(test_embedding).squeeze().cpu().numpy()

            train_target = train_y_long[:, i].cpu().numpy()
            test_target = test_y_long[:, i].cpu().numpy()

            train_mse = mean_squared_error(train_target, train_pred)
            test_mse = mean_squared_error(test_target, test_pred)

            train_r2 = r2_score(train_target, train_pred)
            test_r2 = r2_score(test_target, test_pred)

            print(f'Embedding: {train_embedding.shape}')
            print(f'Property: {property_name}')
            print(f'  Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}')
            print(f'  Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}')

            #add the results to the dictionary
            name_of_embedding = embeddings_names[ii]
            results[(name_of_embedding, property_name)] = (train_mse, test_mse, train_r2, test_r2)

    ii += 1

#save results
with open("results/"+DATASET+"_"+MODEL+"_results_limited_cv_long.pkl", "wb") as f:
    pkl.dump(results, f)

In [ ]:

colors_long = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w', 'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        test_r2 = results[(name_of_embedding, property_names_long)][3]
        if test_r2 < -0.05:  # Handle negative R² values
            test_r2 = -0.05
        x_points.append(j)
        y_points.append(test_r2)
    
    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/"+DATASET+"_"+MODEL+"_test_R2_plot_long.png', dpi=300, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12, 8))

for i, property_names_long in enumerate(property_names_long):
    x_points = []
    y_points = []
    for j, embedding in enumerate(embeddings):
        name_of_embedding = embeddings_names[j]
        train_r2 = results[(name_of_embedding, property_names_long)][2]
        if train_r2 < -0.05:  # Handle negative R² values
            train_r2 = -0.05
        x_points.append(j)
        y_points.append(train_r2)

    # Plotting the line for the current property
    plt.plot(x_points, y_points, label=property_names_long, color=colors_long[i], marker='x')

plt.xticks(range(len(embeddings)), embeddings_names)
plt.xlabel('Embedding')
plt.ylabel('R²')
plt.legend()
plt.title('FC matrice - GCN - R² for different embeddings and properties')
plt.show()

#save the plot
plt.savefig('results/"+DATASET+"_"+MODEL+"_train_R2_plot_long.png', dpi=300, bbox_inches='tight')